In [1]:
from transformers import pipeline
import glob
from PIL import Image
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from collections import Counter
import warnings
from sklearn.metrics import classification_report
import torch
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import random
import pandas as pd
import glob
from tqdm import tqdm
warnings.filterwarnings("ignore")

extractor = AutoFeatureExtractor.from_pretrained("Devarshi/Brain_Tumor_Detector_swin")
model = AutoModelForImageClassification.from_pretrained("Devarshi/Brain_Tumor_Detector_swin")
# pipe = pipeline("image-classification", model="surajjoshi/swin-tiny-patch4-window7-224-finetuned-brainTumorData")

In [7]:
file = "archive\Brain Tumor\Brain Tumor"
df = pd.read_csv("archive\Brain Tumor.csv")
prd_list = []
true_list = []
for filename in tqdm(glob.glob(file + "/*.jpg")):
    name = re.findall(r'Image+\d+', str(filename))
    true_list.append(df[df["Image"] == name[0]]['Class'].values[0])
    # image = Image.open(filename)
    image = cv.imread(filename)
    # 糊糊化(blur)
    blur = cv.GaussianBlur(image, (7,7), cv.BORDER_DEFAULT)
    # 輪廓化(edge cascade)
    canny = cv.Canny(blur, 125, 175)
    # 膨脹 dilating
    dilated = cv.dilate(canny, (7,7), iterations=3)
    # 侵蝕 eroding
    eroding = cv.erode(canny, (3,3), iterations=1)
    # cv.waitKey(0)
    image = extractor(image, return_tensors="pt")
    logits = model(**image).logits
    predicted_label = logits.argmax(-1).item()
    prd_list.append(model.config.id2label[predicted_label])
true_list_tras = ["no" if x == 0 else "yes" for x in true_list]
report = classification_report(true_list_tras, prd_list)
print(report)
# print(Counter(prd_list))

100%|██████████| 3762/3762 [33:49<00:00,  1.85it/s]

              precision    recall  f1-score   support

          no       0.65      0.56      0.60      2079
         yes       0.54      0.63      0.58      1683

    accuracy                           0.59      3762
   macro avg       0.59      0.60      0.59      3762
weighted avg       0.60      0.59      0.59      3762



In [17]:
true_list = []
prd_list = []
for i,filename in enumerate(dataset["test"][1:]):
    image = dataset["test"][i]["image"]
    lable = dataset["test"][i]['objects']["category"][0]
    image = extractor(image, return_tensors="pt")
    logits = model(**image).logits
    predicted_label = logits.argmax(-1).item()
    prd_list.append(model.config.id2label[predicted_label])
    true_list.append(lable)
print(Counter(prd_list))
print(Counter(true_list))

Counter({'yes': 4, 'no': 1})
Counter({2: 3, 1: 2})
